In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore

cred = credentials.Certificate("govmadad-firebase-adminsdk-fbsvc-8999227f8b.json")
firebase_admin.initialize_app(cred)

In [12]:
db = firestore.client()
docs = db.collection(u'complaints').stream()


In [13]:
import pandas as pd
rows = [doc.to_dict() for doc in docs]
df = pd.DataFrame(rows)
print(df.head())

              Phone     Status Pincode  \
0  Fetched From UID  Forwarded  208001   
1  Fetched From UID    Pending  208001   
2  Fetched From UID  Forwarded  208001   
3  Fetched From UID    Pending  208001   
4  Fetched From UID    Pending  208001   

                                            Response     Area  \
0                                    Will be Solved    Kanpur   
1  I've categorized your complaint as an emergenc...   Kanpur   
2                               We're working on it.   Kanpur   
3  Your complaint is registered with the Police d...  Lucknow   
4  I'm happy to help! However, I notice that the ...   Kanpur   

          Subcategory               Complaint  \
0   Fare Overcharging                   Crime   
1  Road Safety Issues  Crime near the society   
2        Power Outage           Leaking water   
3  Contaminated Water                   Crime   
4    Blocked Drainage                 Nothing   

                        Department RemainingDays PredictedTim

In [ ]:
def df_to_markdown(df):
    return df.to_markdown(index=False)

table_text = df_to_markdown(df)

In [17]:
print(table_text)

| Phone            | Status    |   Pincode | Response                                                                                                                                                                                                                                                                                          | Area    | Subcategory                        | Complaint              | Department                        | RemainingDays   | PredictedTime   | Urgency   | ComplaintDate                    | Date       | Category          | FiledBy          |           UID | ImageCaption                                                                 |
|:-----------------|:----------|----------:|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [18]:
from langchain.schema import Document

doc = Document(page_content=table_text, metadata={"source": "firebase_table"})

In [ ]:
import os
from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
os.environ["GOOGLE_API_KEY"]=os.getenv("GOOGLE_API_KEY")
llm = ChatGroq(groq_api_key=groq_api_key, model_name="llama-3.1-8b-instant")

In [24]:
def create_department_doc(df, department):
    filtered_df = df[df['department'] == department]
    if filtered_df.empty:
        return None
    markdown = df_to_markdown(filtered_df)
    return Document(page_content=markdown, metadata={"department": department})

In [26]:
def get_qa_chain(doc):
    vectorstore = FAISS.from_documents([doc], GoogleGenerativeAIEmbeddings(model = "models/embedding-001"))
    retriever = vectorstore.as_retriever()
    qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
    return qa

In [ ]:
def ask():
    data = request.json
    department = data.get('department')
    question = data.get('question')

    if not department or not question:
        return jsonify({"error": "Please provide both 'department' and 'question'"}), 400

    df = fetch_data()
    doc = create_department_doc(df, department)

    if not doc:
        return jsonify({"error": f"No data found for department '{department}'"}), 404

    qa = get_qa_chain(doc)
    answer = qa.run(question)

    return jsonify({"answer": answer})


In [27]:
df = pd.read_csv("complaints.csv")
df.head(5)

,Phone,Status,Pincode,Response,Area,Subcategory,Complaint,Department,RemainingDays,PredictedTime,Urgency,ComplaintDate,Date,Category,FiledBy,UID,ImageCaption
0,Fetched From UID,Forwarded,208001,Will be Solved,Kanpur,Fare Overcharging,Crime,the Police Department,0,33 days,YES,2025-02-14 18:59:36.259000+00:00,2025-02-15,Electricity Issue,Fetched From UID,NaN,NaN
1,Fetched From UID,Pending,208001,I've categorized your complaint as an emergenc...,Kanpur,Road Safety Issues,Crime near the society,the Police Department,0,33 days,YES,2025-02-15 14:03:01.573000+00:00,2025-02-15,Crime,Fetched From UID,2.222222e+09,NaN
2,Fetched From UID,Forwarded,208001,We're working on it.,Kanpur,Power Outage,Leaking water,Public Works Department (PWD),0,33 days,YES,2025-02-14 21:13:25.540000+00:00,2025-02-04,Electricity Issue,Fetched From UID,2.220000e+02,NaN
3,Fetched From UID,Pending,208001,Your complaint is registered with the Police d...,Lucknow,Contaminated Water,Crime,the Police department,33 days,33 days,YES,2025-04-25 16:35:07.377000+00:00,2025-04-16,Crime,Fetched From UID,2.222222e+09,several people standing on a stage with a proj...
4,Fetched From UID,Pending,208001,"I'm happy to help! However, I notice that the ...",Kanpur,Blocked Drainage,Nothing,the Cleaning & Welfare Ministry,0,22 days,NO,2025-02-15 14:12:03.567000+00:00,NaN,Road Maintenance,Fetched From UID,2.222222e+09,arafed pile of garbage on the side of the road


In [31]:
df[df['Department'] == 'the Police Department'].head(5)

,Phone,Status,Pincode,Response,Area,Subcategory,Complaint,Department,RemainingDays,PredictedTime,Urgency,ComplaintDate,Date,Category,FiledBy,UID,ImageCaption
0,Fetched From UID,Forwarded,208001,Will be Solved,Kanpur,Fare Overcharging,Crime,the Police Department,0,33 days,YES,2025-02-14 18:59:36.259000+00:00,2025-02-15,Electricity Issue,Fetched From UID,NaN,NaN
1,Fetched From UID,Pending,208001,I've categorized your complaint as an emergenc...,Kanpur,Road Safety Issues,Crime near the society,the Police Department,0,33 days,YES,2025-02-15 14:03:01.573000+00:00,2025-02-15,Crime,Fetched From UID,2.222222e+09,NaN
7,Fetched From UID,Pending,208001,Your complaint is registered with the Police D...,Kanpur,Fare Overcharging,Crime,the Police Department,0,33 days,YES,2025-02-14 21:15:34.381000+00:00,2025-02-06,Road Maintenance,Fetched From UID,2.220000e+02,the logo of the institute of engineering and t...
